In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

df_train = pd.read_csv('train_toxic.csv')
print(df_train.head(10))

print("\n\n\n")

df_test = pd.read_csv("test_toxic.csv")
print(df_test.head(10))


df_mixed_train = pd.read_csv("train_toxic_mixed.csv")
print(df_test.head(10))

                                                text is_toxic
0  Yine de Pestagyang kadar adım atarak Gestapo t...      yes
1  Kendimi aptal hissettirdi ya günlerce delirdim...      yes
2  Hakkında hayırlısı olsun Bul birini kuttulak s...      yes
3  Makaleyi yeniden adlandır Bu makalenin adı Alb...       no
4  Atların dişleri yaşlandıkça uzar ve atın gerçe...       no
5      delii ayyaş işte ayık olsa normal dusunecekte      yes
6                     Çilesiz bir günüm olmadı gitti       no
7      ne kıvırcık ne kısa ezelin kör anası mısın aq      yes
8             Bunların topu yakacan bunlar tam kahpe      yes
9  lan dur amk rumu zaten ortalık karışık sen zat...      yes




                                                text is_toxic
0  Lan siktirin gidin AMK pozitif sik kafaları Ül...      yes
1  Kabul Dream Chaser programı ile ilgili olmayan...       no
2  Thanx Teşekkürler Bay horoz Bana son bir şans ...       no
3                Zeka yaşı falan da orası ayrı sjsjd      yes
4  L

In [2]:
x_train = df_train.text
y_train = df_train.is_toxic

x_test= df_test.text
y_test = df_test.is_toxic

x_mixed_train = df_mixed_train.text
y_mixed_train = df_mixed_train.is_toxic

x_mixed_test= df_test.text
y_test = df_test.is_toxic


In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, base ------")
my_labels = ["yes", "no"]
print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_base_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_base_report)

------Naive Bayes, base ------
accuracy 0.875
              precision    recall  f1-score   support

         yes       0.86      0.89      0.88       995
          no       0.89      0.86      0.87      1005

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.87      2000
weighted avg       0.88      0.88      0.87      2000



In [5]:
df_mixed_default_train = pd.read_csv("train_toxic_default_mixed.csv")


x_mixed_default_train = df_mixed_default_train.text
y_mixed_default_train = df_mixed_default_train.is_toxic
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_default_train, y_mixed_default_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_test)

print("------Naive Bayes, data augmented with default gpt2------")

print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_default_augmented_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_default_augmented_report)


------Naive Bayes, data augmented with default gpt2------
accuracy 0.805
              precision    recall  f1-score   support

         yes       0.72      0.99      0.84       995
          no       0.99      0.62      0.76      1005

    accuracy                           0.81      2000
   macro avg       0.86      0.81      0.80      2000
weighted avg       0.86      0.81      0.80      2000



In [6]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(x_mixed_train, y_mixed_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(x_mixed_test)

print("------Naive Bayes, data augmented with our model------")

print('accuracy %s' % accuracy_score(y_pred, y_test))
NB_augmented_report = classification_report(y_test, y_pred,target_names=my_labels)
print(NB_augmented_report)


------Naive Bayes, data augmented with our model------
accuracy 0.825
              precision    recall  f1-score   support

         yes       0.74      0.99      0.85       995
          no       0.99      0.66      0.79      1005

    accuracy                           0.82      2000
   macro avg       0.87      0.83      0.82      2000
weighted avg       0.87      0.82      0.82      2000



In [9]:
print("base\n")
print(NB_base_report)
print("------------------------------------------")
print("data augmented with baseline gpt2\n")
print(NB_default_augmented_report)
print("------------------------------------------")

print("data augmented with our model\n")
print(NB_augmented_report)

base

              precision    recall  f1-score   support

         yes       0.86      0.89      0.88       995
          no       0.89      0.86      0.87      1005

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.87      2000
weighted avg       0.88      0.88      0.87      2000

------------------------------------------
data augmented with baseline gpt2

              precision    recall  f1-score   support

         yes       0.72      0.99      0.84       995
          no       0.99      0.62      0.76      1005

    accuracy                           0.81      2000
   macro avg       0.86      0.81      0.80      2000
weighted avg       0.86      0.81      0.80      2000

------------------------------------------
data augmented with our model

              precision    recall  f1-score   support

         yes       0.74      0.99      0.85       995
          no       0.99      0.66      0.79      1005

    accuracy            